In [1]:
import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow.keras.utils import Sequence
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from sklearn.model_selection import train_test_split
from PIL import Image
import os
import numpy as np


In [2]:
def get_available_gpus():
    """ Retorna uma lista dos identificadores das GPUs disponíveis. """
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

available_gpus = get_available_gpus()
if available_gpus:
    print("GPUs disponíveis:", available_gpus)
else:
    print("Nenhuma GPU disponível.")


GPUs disponíveis: ['/device:GPU:0']


In [9]:
class UCMercedDataset(Sequence):
    def __init__(self, root_dir, image_size=(256, 256)):
        self.root_dir = root_dir
        self.image_size = image_size
        self.images = []
        self.labels = []
        
        for label_dir in os.listdir(root_dir):
            class_dir = os.path.join(root_dir, label_dir)
            if os.path.isdir(class_dir):
                for img_file in os.listdir(class_dir):
                    if img_file.endswith(".tif") or img_file.endswith(".jpg"):
                        self.images.append(os.path.join(class_dir, img_file))
                        self.labels.append(label_dir)
        self.labels = [self.labels.index(l) for l in self.labels]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).resize(self.image_size).convert('RGB')
        label = self.labels[idx]
        image = np.array(image) / 255.0
        return image, label

    def on_epoch_end(self):
        temp = list(zip(self.images, self.labels))
        np.random.shuffle(temp)
        self.images, self.labels = zip(*temp)


In [10]:

dataset = UCMercedDataset(root_dir='Data/UCM_captions')
images, labels = zip(*[dataset[i] for i in range(len(dataset))])  # Extrai todas as imagens e labels
images_label_dict = dict(zip(images, labels))  # Cria um dicionário com imagens e labels
# Divisão de dados
train_images, test_images, train_labels, test_labels = train_test_split(
    images, labels, test_size=0.20, random_state=0
)

np.random.shuffle(images_label_dict)

# Criar objetos Dataset do TensorFlow
train_data = tf.data.Dataset.from_tensor_slices((list(train_images), list(train_labels))).batch(32)
test_data = tf.data.Dataset.from_tensor_slices((list(test_images), list(test_labels))).batch(32)

print("Tamanho do conjunto de treino:", len(train_images))
print("Tamanho do conjunto de teste:", len(test_images))


KeyboardInterrupt: 

In [ ]:

# Carrega o modelo VGG16 sem pesos pré-treinados
base_model = VGG16(weights=None, include_top=False, input_shape=(256, 256, 3))
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = Dense(21, activation='softmax')(x)  # 21 classes

model = Model(inputs=base_model.input, outputs=x)

print(model.summary())


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [ ]:
# Diretório para salvar os checkpoints do modelo
checkpoint_dir = 'model_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_filepath = os.path.join(checkpoint_dir, 'best_model.h5')

# Callback para salvar o modelo com a melhor acurácia de validação
model_checkpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

# Callback para parar o treinamento se a acurácia de validação não melhorar após 5 épocas
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    verbose=1,
    mode='max',
    restore_best_weights=True
)

# Callback para TensorBoard
tensorboard_callback = TensorBoard(
    log_dir='logs',
    histogram_freq=1,  # Frequência (em épocas) para calcular histogramas de ativações e pesos
    write_graph=True
)

# Lista de callbacks
callbacks_list = [model_checkpoint, early_stopping, tensorboard_callback]


In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Treinamento
history = model.fit(
    train_data,  # passando diretamente o dataset de treino
    validation_data=test_data,  # passando diretamente o dataset de teste
    epochs=3,
    callbacks=callbacks_list,
    verbose=1  # Para obter detalhes do treinamento em cada época
)

print("Treinamento concluído")


Epoch 1/50
53/53 [==============================] - ETA: 0s - loss: 0.5776 - accuracy: 0.9810
Epoch 1: val_accuracy improved from -inf to 1.00000, saving model to model_checkpoints\best_model.h5
53/53 [==============================] - 27s 431ms/step - loss: 0.5776 - accuracy: 0.9810 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/50
53/53 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 2: val_accuracy did not improve from 1.00000
53/53 [==============================] - 20s 386ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/50
53/53 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3: val_accuracy did not improve from 1.00000
53/53 [==============================] - 20s 381ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/50
53/53 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 